# Adding Geo Spatial Data

### Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

### Scraping into Dataframe

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'html')
table = soup.find_all('table')[0] 
df_ = pd.read_html(str(table))
df = df_[0]

In [3]:
df.rename({"Postcode": "PostalCode"}, axis=1, inplace=True)

In [4]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Excluding the "Not Assigned" Borough

In [5]:
df = df[df["Borough"] != "Not assigned"]

In [6]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Updating Neighborhood with Borough if Neighborhood is "Not Assigned"

In [7]:
df[df["Neighbourhood"] == "Not assigned"]

,PostalCode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


In [8]:
df["Neighbourhood"] = np.where(df["Neighbourhood"] == "Not assigned", df["Borough"], df["Neighbourhood"])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [10]:
df[df["Borough"] == "Queen's Park"]

,PostalCode,Borough,Neighbourhood
9,M9A,Queen's Park,Queen's Park


### Maybe, two rows will be combined into one row with the neighborhoods separated with a comma

In [11]:
df_agg = df.groupby(["PostalCode", "Borough"]).agg({'Neighbourhood': lambda x: ', '.join(x)}).reset_index()

In [12]:
df_agg.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df[df["PostalCode"] == "M5B"]

,PostalCode,Borough,Neighbourhood
16,M5B,Downtown Toronto,Ryerson
17,M5B,Downtown Toronto,Garden District


In [14]:
df_agg[df_agg["PostalCode"] == "M5B"]

,PostalCode,Borough,Neighbourhood
54,M5B,Downtown Toronto,"Ryerson, Garden District"


### Shape of Dataframe

In [15]:
df_agg.shape

(103, 3)

### Geo Spatial Data

In [16]:
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")

In [17]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_geo.rename({"Postal Code": "PostalCode"}, axis=1, inplace=True)

In [19]:
df_agg_geo = pd.merge(df_agg,
                      df_geo,
                      on="PostalCode",
                      how="left")

In [20]:
df_agg_geo.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
